# Evolver Loop 2 - LB Feedback Analysis

## Submission Result
- CV: 70.7343
- LB: 70.7343 (gap: 0.0000)
- Target: 68.931058
- Gap to target: 1.803 points (2.6%)

## Key Insight
The CV-LB gap is ZERO! This is a pure optimization problem with no distribution shift.
The scoring is deterministic and exact.

## Critical Finding from Previous Analysis
- Ensemble with overlaps scores **67.77** (BELOW target!)
- Valid ensemble (no overlaps) scores **70.73** (same as baseline)
- **ALL improvement is locked behind overlaps**
- 30 N values have overlaps with 2.96 points of improvement potential

In [1]:
import numpy as np
import pandas as pd
import math
from numba import njit
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import random

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

tx, ty = make_polygon_template()
print('Template loaded')

Template loaded


In [2]:
# Overlap detection functions
def get_shapely_polygon(cx, cy, deg, tx, ty):
    r = deg * np.pi / 180.0
    c = np.cos(r)
    s = np.sin(r)
    px = c * tx - s * ty + cx
    py = s * tx + c * ty + cy
    return Polygon(zip(px, py))

def has_overlap(xs, ys, degs, tx, ty):
    n = len(xs)
    if n <= 1:
        return False
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def find_overlapping_pairs(xs, ys, degs, tx, ty):
    n = len(xs)
    if n <= 1:
        return []
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    pairs = []
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx <= i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                pairs.append((i, idx))
    return pairs

print('Overlap detection loaded')

Overlap detection loaded


In [3]:
# Load the ensemble with overlaps (67.77 score)
ensemble_path = '/home/nonroot/snapshots/santa-2025/21108486172/code/experiments/001_baseline/ensemble_submission.csv'
df_ensemble = pd.read_csv(ensemble_path)
df_ensemble['N'] = df_ensemble['id'].astype(str).str.split('_').str[0].astype(int)

# Load baseline (70.73 score, no overlaps)
baseline_path = '/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv'
df_baseline = pd.read_csv(baseline_path)
df_baseline['N'] = df_baseline['id'].astype(str).str.split('_').str[0].astype(int)

print('Loaded ensemble and baseline')

Loaded ensemble and baseline


In [4]:
# Analyze the overlap situation for each N
overlap_info = []

for n in range(1, 201):
    g_ens = df_ensemble[df_ensemble['N'] == n]
    g_base = df_baseline[df_baseline['N'] == n]
    
    xs_ens = strip(g_ens['x'].to_numpy())
    ys_ens = strip(g_ens['y'].to_numpy())
    ds_ens = strip(g_ens['deg'].to_numpy())
    
    xs_base = strip(g_base['x'].to_numpy())
    ys_base = strip(g_base['y'].to_numpy())
    ds_base = strip(g_base['deg'].to_numpy())
    
    score_ens = score_group(xs_ens, ys_ens, ds_ens, tx, ty)
    score_base = score_group(xs_base, ys_base, ds_base, tx, ty)
    
    pairs = find_overlapping_pairs(xs_ens, ys_ens, ds_ens, tx, ty)
    
    overlap_info.append({
        'n': n,
        'score_ens': score_ens,
        'score_base': score_base,
        'improvement': score_base - score_ens,
        'num_overlaps': len(pairs),
        'has_overlap': len(pairs) > 0
    })

df_info = pd.DataFrame(overlap_info)
print(f'Total ensemble score: {df_info["score_ens"].sum():.6f}')
print(f'Total baseline score: {df_info["score_base"].sum():.6f}')
print(f'Potential improvement: {df_info["improvement"].sum():.6f}')
print(f'\nN values with overlaps: {df_info["has_overlap"].sum()}')
print(f'N values without overlaps: {(~df_info["has_overlap"]).sum()}')

Total ensemble score: 67.772662
Total baseline score: 70.734327
Potential improvement: 2.961665

N values with overlaps: 30
N values without overlaps: 170


In [5]:
# Show N values with overlaps sorted by improvement potential
overlap_df = df_info[df_info['has_overlap']].sort_values('improvement', ascending=False)
print('N values with overlaps (sorted by improvement potential):')
print(overlap_df[['n', 'score_ens', 'score_base', 'improvement', 'num_overlaps']].head(20).to_string())

N values with overlaps (sorted by improvement potential):
     n  score_ens  score_base  improvement  num_overlaps
6    7   0.162010    0.399897     0.237887            21
5    6   0.173625    0.399610     0.225985            15
9   10   0.166604    0.376630     0.210026            41
8    9   0.178013    0.387415     0.209402            34
4    5   0.212694    0.416850     0.204155            10
7    8   0.187564    0.385407     0.197844            27
3    4   0.228621    0.416545     0.187924             6
11  12   0.198679    0.372724     0.174045            48
14  15   0.214172    0.379203     0.165030            61
16  17   0.208784    0.370040     0.161257            55
2    3   0.305312    0.434745     0.129433             3
10  11   0.257940    0.375736     0.117796            30
17  18   0.274445    0.368771     0.094326            36
21  22   0.294317    0.375258     0.080941            58
22  23   0.290307    0.368752     0.078445            48
20  21   0.302114    0.376451 

In [6]:
# Analyze a specific N value to understand the overlap repair challenge
test_n = 7  # Pick N=7 which has highest improvement potential

g = df_ensemble[df_ensemble['N'] == test_n]
xs = strip(g['x'].to_numpy())
ys = strip(g['y'].to_numpy())
ds = strip(g['deg'].to_numpy())

pairs = find_overlapping_pairs(xs, ys, ds, tx, ty)
print(f'N={test_n}: {len(pairs)} overlapping pairs')
print(f'Score with overlaps: {score_group(xs, ys, ds, tx, ty):.6f}')

# Get baseline score for comparison
g_base = df_baseline[df_baseline['N'] == test_n]
xs_base = strip(g_base['x'].to_numpy())
ys_base = strip(g_base['y'].to_numpy())
ds_base = strip(g_base['deg'].to_numpy())
print(f'Baseline score: {score_group(xs_base, ys_base, ds_base, tx, ty):.6f}')
print(f'Improvement potential: {score_group(xs_base, ys_base, ds_base, tx, ty) - score_group(xs, ys, ds, tx, ty):.6f}')

print('\nOverlapping pairs:')
for i, j in pairs[:10]:
    print(f'  Trees {i} and {j}')

N=7: 21 overlapping pairs
Score with overlaps: 0.162010
Baseline score: 0.399897
Improvement potential: 0.237887

Overlapping pairs:
  Trees 0 and 2
  Trees 0 and 4
  Trees 0 and 3
  Trees 0 and 6
  Trees 0 and 1
  Trees 0 and 5
  Trees 1 and 2
  Trees 1 and 4
  Trees 1 and 3
  Trees 1 and 6


In [7]:
# Try a simple overlap repair: nudge overlapping trees apart
def repair_overlaps_simple(xs, ys, degs, tx, ty, max_iter=5000):
    """Try to repair overlaps by nudging trees apart"""
    xs = xs.copy()
    ys = ys.copy()
    
    step_sizes = [0.01, 0.005, 0.002, 0.001, 0.0005, 0.0002, 0.0001]
    directions = [(1,0), (-1,0), (0,1), (0,-1), (1,1), (1,-1), (-1,1), (-1,-1)]
    
    for iteration in range(max_iter):
        pairs = find_overlapping_pairs(xs, ys, degs, tx, ty)
        if not pairs:
            return xs, ys, True  # Success!
        
        # Pick a random overlapping pair
        i, j = random.choice(pairs)
        
        # Try moving each tree in the pair
        improved = False
        for tree_idx in [i, j]:
            for step in step_sizes:
                for dx, dy in directions:
                    old_x, old_y = xs[tree_idx], ys[tree_idx]
                    xs[tree_idx] += dx * step
                    ys[tree_idx] += dy * step
                    
                    # Check if this tree still overlaps with any other
                    new_pairs = find_overlapping_pairs(xs, ys, degs, tx, ty)
                    if len(new_pairs) < len(pairs):
                        improved = True
                        break  # Keep the move
                    else:
                        # Revert
                        xs[tree_idx], ys[tree_idx] = old_x, old_y
                
                if improved:
                    break
            if improved:
                break
        
        if iteration % 500 == 0:
            print(f'  Iteration {iteration}: {len(pairs)} overlapping pairs')
    
    return xs, ys, False  # Failed to fully repair

print('Repair function defined')

Repair function defined


In [8]:
# Test overlap repair on N=7
print(f'Testing overlap repair on N={test_n}...')
print(f'Initial overlapping pairs: {len(find_overlapping_pairs(xs, ys, ds, tx, ty))}')

xs_repaired, ys_repaired, success = repair_overlaps_simple(xs.copy(), ys.copy(), ds, tx, ty, max_iter=2000)

if success:
    print(f'\nSUCCESS! Overlaps repaired.')
    new_score = score_group(xs_repaired, ys_repaired, ds, tx, ty)
    print(f'New score: {new_score:.6f}')
    print(f'Baseline score: {score_group(xs_base, ys_base, ds_base, tx, ty):.6f}')
else:
    remaining_pairs = find_overlapping_pairs(xs_repaired, ys_repaired, ds, tx, ty)
    print(f'\nFailed to fully repair. Remaining overlapping pairs: {len(remaining_pairs)}')
    new_score = score_group(xs_repaired, ys_repaired, ds, tx, ty)
    print(f'Score after partial repair: {new_score:.6f}')

Testing overlap repair on N=7...
Initial overlapping pairs: 21
  Iteration 0: 21 overlapping pairs


  Iteration 500: 21 overlapping pairs


  Iteration 1000: 21 overlapping pairs


  Iteration 1500: 21 overlapping pairs



Failed to fully repair. Remaining overlapping pairs: 21
Score after partial repair: 0.162010


In [ ]:
# Summary
print('='*60)
print('SUMMARY')
print('='*60)
print(f'Current best LB: 70.7343')
print(f'Target: 68.931058')
print(f'Gap: 1.803 points (2.6%)')
print()
print('KEY INSIGHT:')
print('- Ensemble with overlaps scores 67.77 (BELOW target!)')
print('- Valid ensemble (no overlaps) scores 70.73 (same as baseline)')
print('- ALL 2.96 points of improvement is locked behind overlaps')
print('- 30 N values have overlaps')
print()
print('NEXT STEP:')
print('Implement robust overlap repair to unlock the 2.96 points of improvement')